In [1]:
import json
from azure.eventhub import EventHubProducerClient, EventData
import os
import socket
import random

from random import randrange


eventHubConnString = "<event hub connection string>" 
eventHubNameImpressions = "impressions"
eventHubNameClicks = "clicks"

producer_impressions = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameImpressions)
producer_clicks = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameClicks)

hostname = socket.gethostname()

StatementMeta(, 2459b345-c8bb-452b-ac64-711c3be3c588, 5, Finished, Available)

In [2]:
from faker import Faker

productIds = [707,708,711,712,714,715,716,717,718,722,738,739,742,743,747,748,779,780,781,782,783,784,792,793,794,795,796,797,798,799,800,801,808,809,810,813,822,835,836,838,858,859,860,864,865,867,868,869,870,873,874,875,876,877,880,881,883,884,885,886,889,891,892,893,894,895,896,899,900,904,905,907,908,909,910,913,916,917,918,920,924,925,926,935,936,937,938,939,940,944,945,947,948,949,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,996,997,998,999]


def generateClickEvent(impressionEvent):
    faker = Faker()
    click = {}
    click["clickId"] = faker.uuid4()
    click["impressionId"] = impressionEvent["impressionId"]
    click["clickDate"] = impressionEvent["impressionDate"]
    click["productId"] = impressionEvent["productId"]
    return click


def generateImpressionEvent(isUnsupportedBrowser, isAnomaly, productId):
    faker = Faker()
    impression = {}
    impression["impressionId"] = faker.uuid4()
    impression["impressionDate"] = faker.date_time_between(start_date="-1w", end_date="now").isoformat()
    if productId:
        impression["productId"] = str(productId)
    else:
        impression["productId"] = faker.random_element(productIds)

    if isUnsupportedBrowser:
        impression["browser"] = "unsupported"
    else:
        impression["browser"] = faker.random_element(["Edge", "Chrome", "Safari", "Firefox"])
    impression["browserVersion"] = faker.random_element(["10.2", "13.6", "8.6", "8.5", "11.2", "14.6", "6.6", "4.5"])
    impression["device"] = faker.random_element(["mobile", "computer", "tablet", "mobile", "computer"])
    impression["source"] = faker.random_element( ["organic", "bing", "google", "facebook"])
    impression["ip_address"] = faker.ipv4()
    impression["landing_page"] = faker.uri()
    if isAnomaly:
        impression["page_loading_seconds"] = faker.random_number(4)/100
    else:
        impression["page_loading_seconds"] = faker.random_number(4)/1000
    return impression

StatementMeta(, 2459b345-c8bb-452b-ac64-711c3be3c588, 6, Finished, Available)

In [3]:

def sendToEventsHub(jsonEvent, producer):
    eventString = json.dumps(jsonEvent)
    print(eventString) 
    event_data_batch = producer.create_batch() 
    event_data_batch.add(EventData(eventString)) 
    producer.send_batch(event_data_batch)

def generateEvents(isUnsupportedBrowser = False, isAnomaly = False):
    try:
        while True:
            impressionEvent = generateImpressionEvent(isUnsupportedBrowser, isAnomaly, None)    
            sendToEventsHub(impressionEvent, producer_impressions)
            if random.randint(1, 100) > 80:
                clickEvent = generateClickEvent(impressionEvent)    
                sendToEventsHub(clickEvent, producer_clicks)
    except KeyboardInterrupt:
        producer_impressions.close()
        producer_clicks.close()


StatementMeta(, 2459b345-c8bb-452b-ac64-711c3be3c588, 7, Finished, Available)

In [4]:
generateEvents(False, True)

StatementMeta(, 2459b345-c8bb-452b-ac64-711c3be3c588, 8, Finished, Cancelled)

{"impressionId": "d6a2336b-8ce8-46da-85fa-9c016a0bcfd5", "impressionDate": "2024-01-31T05:47:16.251332", "productId": 944, "browser": "Edge", "browserVersion": "8.5", "device": "mobile", "source": "organic", "ip_address": "189.172.157.176", "landing_page": "http://www.thomas.com/searchauthor.php", "page_loading_seconds": 6.383}
{"impressionId": "1916ee04-48a4-48a0-9346-cd0bfd565d83", "impressionDate": "2024-02-04T13:51:20.559446", "productId": 838, "browser": "Firefox", "browserVersion": "13.6", "device": "computer", "source": "google", "ip_address": "203.191.215.239", "landing_page": "https://mcgee.info/tagauthor.htm", "page_loading_seconds": 1.876}
{"impressionId": "b707732a-ee17-41e2-a2e4-5e9dd506e3c4", "impressionDate": "2024-01-30T02:35:44.686525", "productId": 984, "browser": "Safari", "browserVersion": "10.2", "device": "tablet", "source": "google", "ip_address": "75.85.207.242", "landing_page": "https://www.bell-meyer.info/main/categoriesregister.htm", "page_loading_seconds": 9